# 📌 Xử lý hàng loạt file âm thanh: Lọc nhiễu & Loại bỏ khoảng lặng

## 🔧 Cài đặt các thư viện cần thiết

In [ ]:
!pip install numpy scipy librosa webrtcvad soundfile tqdm

## 🔄 Nhập các thư viện cần thiết

In [ ]:

import os
import numpy as np
import librosa
import soundfile as sf
import webrtcvad
import wave
import struct
from tqdm import tqdm
    

## 🔹 Chuyển đổi âm thanh về PCM 16-bit Mono

In [ ]:

def convert_audio_to_pcm16_mono(input_wav, output_wav, target_sr=16000):
    """
    Chuyển đổi tệp âm thanh về PCM 16-bit mono.
    """
    y, sr = librosa.load(input_wav, sr=target_sr, mono=True)
    sf.write(output_wav, y, target_sr, subtype='PCM_16')
    

## 🔹 Lọc nhiễu bằng Spectral Subtraction

In [ ]:

def spectral_subtraction(y, sr, noise_estimate=None, alpha=1.2):
    """
    Lọc nhiễu trong tín hiệu âm thanh bằng phương pháp Spectral Subtraction.
    """
    n_fft = 2048
    hop_length = 512

    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    magnitude, phase = np.abs(D), np.angle(D)

    if noise_estimate is None:
        noise_frames = int(0.5 * sr / hop_length)
        noise_estimate = np.mean(magnitude[:, :noise_frames], axis=1, keepdims=True)

    magnitude_denoised = np.maximum(magnitude - alpha * noise_estimate, 0)
    D_denoised = magnitude_denoised * np.exp(1j * phase)
    y_denoised = librosa.istft(D_denoised, hop_length=hop_length)

    return y_denoised
    

## 🔹 Loại bỏ khoảng lặng bằng WebRTC VAD

In [ ]:

def remove_silence(input_wav, output_wav, aggressiveness=3):
    """
    Loại bỏ khoảng lặng bằng WebRTC VAD.
    """
    vad = webrtcvad.Vad(aggressiveness)

    with wave.open(input_wav, "rb") as wf:
        sample_rate = wf.getframerate()
        channels = wf.getnchannels()
        width = wf.getsampwidth()

        if channels != 1 or width != 2 or sample_rate not in [8000, 16000, 32000, 48000]:
            print(f"❌ Lỗi: {input_wav} không phải PCM 16-bit mono hoặc có sample rate không hợp lệ!")
            return

        frames = wf.readframes(wf.getnframes())

    frame_duration = 30  # 30ms mỗi frame
    frame_size = int(sample_rate * frame_duration / 1000) * width

    frames = [frames[i:i + frame_size] for i in range(0, len(frames), frame_size)]

    if not frames or len(frames[0]) != frame_size:
        print(f"❌ Lỗi: Kích thước frame không hợp lệ trong {input_wav}")
        return

    voiced_frames = [frame for frame in frames if vad.is_speech(frame, sample_rate)]

    if not voiced_frames:
        print(f"⚠️ Cảnh báo: Không tìm thấy giọng nói trong {input_wav}, giữ nguyên file gốc.")
        return

    with wave.open(output_wav, "wb") as wf_out:
        wf_out.setnchannels(1)
        wf_out.setsampwidth(2)
        wf_out.setframerate(sample_rate)
        for frame in voiced_frames:
            wf_out.writeframes(frame)
    

## 🔹 Xử lý hàng loạt file âm thanh

In [ ]:

def process_audio_folder(input_folder, output_folder):
    """
    Xử lý tất cả các file .wav trong thư mục bằng cách lọc nhiễu và loại bỏ khoảng lặng.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    files = [f for f in os.listdir(input_folder) if f.endswith(".wav")]

    for file in tqdm(files, desc="Processing files"):
        input_path = os.path.join(input_folder, file)
        converted_path = os.path.join(output_folder, "converted_" + file)
        denoised_path = os.path.join(output_folder, "denoised_" + file)
        final_output_path = os.path.join(output_folder, "cleaned_" + file)

        convert_audio_to_pcm16_mono(input_path, converted_path)

        y, sr = librosa.load(converted_path, sr=None)
        y_denoised = spectral_subtraction(y, sr)
        sf.write(denoised_path, y_denoised, sr)

        remove_silence(denoised_path, final_output_path)

        os.remove(converted_path)
        os.remove(denoised_path)

    print(f"✅ Đã xử lý xong tất cả các file! Kết quả lưu tại: {output_folder}")
    

## ▶️ Chạy xử lý âm thanh

In [ ]:

# Đặt thư mục đầu vào và đầu ra
input_directory = "./data/wham/cv"  # Đổi thành thư mục chứa file âm thanh
output_directory = "./data/wham_output"  # Đổi thành thư mục để lưu kết quả

# Chạy xử lý
process_audio_folder(input_directory, output_directory)
    